In [ ]:
import pystac_client
import planetary_computer
import pandas as pd

# 1. Corregimos el warning y configuramos el catálogo
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

def buscar_pares_entrenamiento(bioma_nombre, coords, n_pares=50):
    buffer = 0.1  # Área un poco más grande
    bbox = [coords[0]-buffer, coords[1]-buffer, coords[0]+buffer, coords[1]+buffer]
    
    # A. Buscar imágenes NUBLADAS (Input)
    search_cloudy = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime="2023-01-01/2023-12-31",
        query={"eo:cloud_cover": {"gt": 20, "lt": 60}}, 
        max_items=n_pares
    )
    cloudy_items = list(search_cloudy.items()) # Corregido get_items() -> items()

    pares = []
    for c_item in cloudy_items:
        # B. Por cada nublada, buscamos una LIMPIA (Target) en el mismo sitio
        # Buscamos en un rango de 6 meses alrededor de la foto nublada
        search_clear = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=c_item.bbox,
            datetime="2023-01-01/2023-12-31",
            query={"eo:cloud_cover": {"lt": 5}}, 
            max_items=1
        )
        clear_items = list(search_clear.items())
        
        if clear_items:
            pares.append({
                "bioma": bioma_nombre,
                "id_nublada": c_item.id,
                "id_limpia": clear_items[0].id,
                "nubes_porcentaje": c_item.properties["eo:cloud_cover"],
                "item_nublado": c_item,
                "item_limpio": clear_items[0]
            })
            
    return pares

# Ejemplo de uso para escalar
biomas = {
    "bosque": [-84.0, 10.3],
    "ciudad": [-74.0, 40.7],
    "desierto": [25.0, 25.0],
    "tundra": [68.0, 70.0],
    "sabana": [34.0, -1.3],
    "manglar": [-80.0, 0.5],
    "montaña": [86.9, 27.9],
    "pradera": [-100.0, 44.0],
    "humedal": [-58.0, -34.5],
    "agricultura": [-3.0, 40.0],
    "glaciar": [-72.0, -50.0],
    "volcanico": [-155.0, 19.4],
    "costa": [-77.0, 24.5],
    "isla_tropical": [-60.0, 14.0]
}

dataset_pares = []

for nombre, coords in biomas.items():
    print(f"Buscando pares para {nombre}...")
    dataset_pares.extend(buscar_pares_entrenamiento(nombre, coords, n_pares=100))

df_pares = pd.DataFrame(dataset_pares).drop(columns=['item_nublado', 'item_limpio'])
print(f"\nTotal de pares encontrados: {len(df_pares)}")
display(df_pares.head())


# Ejemplo: Analizar el primer par encontrado
ejemplo = dataset_pares[0]['item_nublado']
props = ejemplo.properties

print(f"Análisis del item: {ejemplo.id}")
print(f"- Nubes totales: {props.get('s2:cloud_shadow_percentage')}%")
print(f"- Nubes Cirrus (finas): {props.get('s2:thin_cirrus_percentage')}%")
print(f"- Sombras: {props.get('s2:cloud_shadow_percentage')}%")

Buscando pares para bosque...
Buscando pares para ciudad...


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import rioxarray
import numpy as np

class CloudSegmentationDataset(Dataset):
    def __init__(self, df_pares, patch_size=256):
        self.df = df_pares
        self.patch_size = patch_size
        # Seleccionamos bandas clave: Rojo, Verde, Azul, Infrarrojo Cercano, SWIR 1 y 2
        self.bands = ["B04", "B03", "B02", "B08", "B11", "B12"]

    def __len__(self):
        return len(self.df)

    def _get_patch(self, item, is_mask=False):
        # Firmar el item para acceso
        signed_item = planetary_computer.sign(item)
        
        if is_mask:
            # Si es máscara, leemos la banda SCL
            url = signed_item.assets["SCL"].href
            da = rioxarray.open_rasterio(url)
            # Recorte central (simplificado para el ejemplo)
            patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
            # Clase 8, 9, 10 son nubes. Clase 3 es sombra.
            mask = np.isin(patch, [3, 8, 9, 10]).astype(np.float32)
            return mask
        else:
            # Si es imagen, leemos las 6 bandas
            band_data = []
            for b in self.bands:
                url = signed_item.assets[b].href
                da = rioxarray.open_rasterio(url)
                patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
                band_data.append(patch)
            
            img = np.concatenate(band_data, axis=0).astype(np.float32)
            return img / 10000.0  # Normalización básica de Sentinel-2

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Obtenemos la imagen nublada y su máscara de nubes (SCL)
        image = self._get_patch(row['item_nublado'], is_mask=False)
        mask = self._get_patch(row['item_nublado'], is_mask=True)
        
        return torch.from_numpy(image), torch.from_numpy(mask)

# Crear el DataLoader
train_ds = CloudSegmentationDataset(df_pares)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rioxarray
import planetary_computer

def visualizar_comparativa(par, patch_size=512):
    # 1. Firmar los items para obtener acceso a los datos
    item_nublado = planetary_computer.sign(par['item_nublado'])
    item_limpio = planetary_computer.sign(par['item_limpio'])
    
    # 2. Definir las bandas RGB (B04, B03, B02) y la máscara (SCL)
    bands_rgb = ["B04", "B03", "B02"]
    
    def get_rgb_data(item):
        data = []
        for b in bands_rgb:
            url = item.assets[b].href
            # Leemos un parche central de 512x512
            da = rioxarray.open_rasterio(url)
            # Calculamos el centro para el recorte
            cy, cx = da.shape[1] // 2, da.shape[2] // 2
            patch = da.isel(y=slice(cy, cy+patch_size), x=slice(cx, cx+patch_size)).values
            data.append(patch)
        
        # Stack y normalización (Sentinel-2 suele verse bien entre 0 y 3000)
        rgb = np.concatenate(data, axis=0).transpose(1, 2, 0)
        return np.clip(rgb / 3000, 0, 1) 

    def get_scl_mask(item):
        url = item.assets["SCL"].href
        da = rioxarray.open_rasterio(url)
        cy, cx = da.shape[1] // 2, da.shape[2] // 2
        patch = da.isel(y=slice(cy, cy+patch_size), x=slice(cx, cx+patch_size)).values[0]
        return patch

    # 3. Cargar los datos
    img_nublada = get_rgb_data(item_nublado)
    img_limpia = get_rgb_data(item_limpio)
    mask_scl = get_scl_mask(item_nublado)

    # 4. Crear la visualización
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    
    ax[0].imshow(img_nublada)
    ax[0].set_title(f"Input: Nublado ({par['nubes_porcentaje']:.1f}%)")
    ax[0].axis('off')
    
    ax[1].imshow(img_limpia)
    ax[1].set_title("Target: Limpio (Ground Truth)")
    ax[1].axis('off')
    
    # Para la máscara SCL usamos un mapa de colores que resalte nubes
    # Nubes (8,9,10) se verán en blanco/amarillo, sombras (3) en oscuro
    ax[2].imshow(mask_scl, cmap='terrain')
    ax[2].set_title("Máscara SCL (Segmentación)")
    ax[2].axis('off')

    plt.tight_layout()
    plt.show()

# Visualizar el primer par del dataset que creaste antes
visualizar_comparativa(dataset_pares[8])